# Gradient Boosting
# Overview

## 1. Introduction
### 1.1 Definition of Gradient Boosting
Gradient Boosting is a machine learning algorithm that belongs to the ensemble learning category. It combines multiple weak models, usually decision trees, to create a single, strong prediction model. The algorithm works by iteratively adding new trees to the model and adjusting their weights based on the gradient of the loss function. The objective of the algorithm is to minimize the loss function, which measures the difference between the actual values and the predicted values. The final prediction model is a weighted sum of all the trees in the ensemble, where the weights are determined based on the optimization of the loss function. Gradient Boosting has proven to be effective for various real-world problems, such as classification and regression tasks, due to its ability to handle complex non-linear relationships and its high accuracy.

### 1.2 Overview of ensemble learning
Ensemble learning is a machine learning technique that involves combining multiple models to form a single, stronger prediction model. The idea behind ensemble learning is that a group of weak models can be combined to form a strong model that outperforms any individual model. Ensemble learning can be used for both classification and regression problems.

There are several types of ensemble learning methods, including:

- Bagging (Bootstrap Aggregating): involves training multiple models on different samples of the training data, with each sample being randomly drawn with replacement. The final prediction is obtained by averaging (or voting) the predictions of the individual models. (such as Random Forest algorithm that we discussed previous notebook)
- Boosting: involves training multiple models in a sequential manner, where each new model aims to correct the mistakes made by the previous model. The final prediction is obtained by weighted sum of all the individual models. Gradient Boosting is a type of boosting algorithm.

<div style="width:image width px; font-size:80%; text-align:center;"><img src='images/Boosting.png' alt="alternate text" title="Boosting" width="width" height="height" style="width:500px;height:250px;" />  </div>
<center> Image: Simulation for boosting technique </center>

- Stacking: involves training multiple models on the same training data and using their predictions as inputs to train a final meta-model. The final prediction is obtained by using the predictions of the meta-model.

## 2. Theoretical foundations of Gradient Boosting

### 2.1 Introduction to decision trees
Decision trees are a fundamental component of Gradient Boosting. They are simple yet powerful machine learning models used for both classification and regression tasks. The main idea behind decision trees is to split the data into smaller and smaller subsets, where each split is based on a single feature of the data. The end result is a tree-like structure that represents a series of decisions based on the features of the data.

Each node in the tree represents a decision based on the value of a feature, and each branch of the tree represents the outcome of that decision. The final prediction is obtained by starting at the root node and following the branches of the tree based on the values of the features, until a leaf node is reached. The prediction is given by the value assigned to the leaf node.
Decision trees are simple to understand, interpret, and visualize, making them a popular choice for many real-world problems. They are also robust to outliers, missing values, and noisy data, and can handle both categorical and numerical data.

In mathematics, a tree can be formally expressed as

$$T(x, \Theta) = \sum_{j=1}^{J} \gamma_j I(x \in R_j)$$

with parameters $\Theta = \{R_j, \gamma_j\}_1^J$. $J$ is usually treated as a meta-parameter. The parameters are found by minimizing the empirical risk

$$\hat{\Theta} = \arg \min_{\Theta} \sum_{j=1}^J \sum_{x_i \in R_j} L(y_i, \gamma_j) = \arg \min_{\Theta} \sum_{j=1}^J L(y_i, T(x_i, \Theta))$$

where a tree partition the space of all joint predictor variable values into disjoint regions $R_j, j=1,2,...,J$, as represented by the terminal nodes of the tree. A constant $\gamma_j$ is assigned to each such region and the predictive rule is

$$x \in R_j \Rightarrow f(x) = \gamma_j$$

- $N$ is the total number of observations
- $L$ is the loss function
- $x_i$ is $i-th$ observation
- $y_i$ is label of $x_i$ in datasets

### 2.2 Mathematical formulation of Gradient Boosting

In the context of Gradient Boosting, decision trees are used as weak models, which are combined to form a single, strong model. The algorithm works by iteratively adding new trees to the model and adjusting their weights based on the **gradient of the loss function**. The objective of the algorithm is to minimize the loss function, which measures the difference between the actual values and the predicted values.

The boosted tree model is a sum of such trees

$$f_M(x) = \sum_{m=1}^{M} T(x, \Theta_m)$$

At each step in the forward stagewise procedure one must solve 

$$\hat{\Theta}_m = \arg \min_{\Theta_m} \sum_{i=1}^{N} L(x_i, f_{m-1}(x_i) + T(x_i, \Theta_m))$$

for the region set and constants $\Theta_m = \{R_{jm}, \gamma_{jm}\}$ of the next tree, given the current model $f_{m-1}(x)$ and 

$$\hat{\gamma}_{jm} = \arg \min_{\gamma_{jm}} \sum_{x \in R_{jm}} L(y_i, f_{m-1}(x_i) + \gamma_{jm})$$

#### Numerical Optimization via Gradient Descent
The loss in using $f(x)$ to predict $y$ on the training data is

$$L(f) = \sum_{i=1}^{N} L(y_i, f(x_i))$$

or you need to find

$$\hat{f} = \arg \min_{f} L(f)$$

where the "paramters" $f \in \mathbb{R}^N$. Numerical optimization procedures solve the above as a sum of component vectors

$$f_m = \sum_{m=0}^{M} h_m$$

where $\_m \in \mathbb{R}^N, f_0 = h_0$ is an initial guess, and each successive $f_m$ is induced based on the current parameter vector $f_{m-1}$, which is the sum of the previous induced update. Steepest descent chooses $h_m = -\lambda_m g_m$ where $\lambda_m$ is a scalar and $g_m \in \mathbb{R}^N$ is the gradient of $L(f)$ evaluated at $f = f_{m-1}$

$$g_m = \left[ \frac{\partial L(y, f(x))}{\partial f(x)} \right]_{f(x) = f_{m-1}(x)}$$

The *step length (learning rate)* $\lambda_m$ is the solution to 

$$\lambda_m = \arg \min_{\lambda} L(f_{m-1} - \lambda g_m)$$

The current solution is then updated 

$$f_m = f_{m-1} - \lambda_m g_m$$

**Note:** In practice, we often fix $\lambda_m$ by specific scaler (e.g $0.3, 0.5, 1$)

## 3. Implementation

From **Theoretical foundations of Gradient Boosting**, we have pseudo-code for regression problem.


1. Initialize $f_0 (x) = \arg \min_{\gamma} \sum_{i=1}^{N} L(y_i, \gamma)$

2. For $m=1$ to $M$:
    
    - (a) For $i=1,2,..., N$ compute 
    $$r_{im} = - \left[ \frac{\partial L(y_i, f(x_i))}{\partial f(x_i)} \right]_{f = f_{m-1}}$$
    
    - (b) Fit a regression tree to targets $r_{im}$ giving terminal regions $R_{jm}, j = 1, 2, ...., J_m$
    
    - (c) For $j = 1, 2, ...., J_m$ compute
    $$\gamma_{jm} = \arg \min_{\gamma} \sum_{x_i \in R_{jm}} L(y_i, f_{m-1}(x_i) + \gamma)$$
    
    - (d) Update $f_m(x) = f_{m-1}(x) + \lambda_m \sum_{j=1}^{J_m} \gamma_{jm}I(x \in R_{jm})$
    
3. Output $\hat{f}(x) = f_M (x)$

In this implementation, we will use square error loss

$$L(y_i, f(x_i)) = (y_i - f(x_i))^2$$

$$\Rightarrow - \frac{\partial L(y_i, f(x_i))}{\partial f(x_i)} = y_i - f(x_i)$$

and the solution for (c) is the **mean** of residuals of (a).

In [1]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor


class GradientBoosting():
    """Implement Gradent Boosting algorithm for regression problem.
    
    Paramters:
        n_estimators (int): number of base models, default 100
        lambd (float): learning rate, default 1.0
    """
    def __init__(self, n_estimators=100, lambd=1.0):
        self.n_estimators = n_estimators
        self.lambd = lambd
        self.trees = []
        
    def _initialize(self, y):
        """Initialize f0 from all labels y."""
        n_observations = len(y)
        self.init_gamma = np.mean(y)
        f0 = np.full((n_observations, ), self.init_gamma)
        return f0
        
    def _compute_gamma(self, residuals, leaf_ids):
        """Compute the optimal gamma.
        
        Args:
            residuals: error 
            leaf_ids: indices of leaves corresponding observations
            
        Return:
            gamma: the optimal value for gamma
        """
        leaves = np.unique(leaf_ids)
        gamma = np.zeros(residuals.shape)
        for leaf in leaves:
            ids = np.where(leaf_ids == leaf)[0]
            gamma[ids] = np.mean(residuals[ids])
            
        return gamma
        
    def fit(self, X, y):
        """Training model from given datasets.
        
        Args:
            X (numpy.array): Array of feature
            y (numpy.array): Array of output 
        """
        # Step 1 - Initialize f0
        self.f0 = self._initialize(y)
        # Assign current f by f0
        f_current = self.f0
        
        # Step 2 - Loop M times
        for i in range(self.n_estimators):
            # Step 2 (a) - Calculate residuals
            residuals = y - f_current
            
            # Step 2 (b) - Fit a regression tree to residuals
            clf = DecisionTreeRegressor()
            clf.fit(X, residuals)
            leaf_ids = clf.apply(X)    # Get terminal region R_j
            
            # Step 2 (c) - Compute gamma_j corresponding R_j
            gamma = self._compute_gamma(residuals, leaf_ids)
            
            # Step 2 (d) - Update
            f_current += self.lambd * gamma
            
            # Append base model to predict in the future
            self.trees.append(clf)
            
            
    def predict(self, X_new):
        """Predict from given input.
        
        Args:
            X_new (numpy.array): given input that need to be predicted
            
        Return:
            preds: the prediction for given X_new from Gradient Boosting algorithm
        """
        # Assign preds by f0 as in the training
        size = X_new.shape[0]
        preds = np.full((size,), self.init_gamma)
        
        # Update the preds from model 1 to model M
        for tree in self.trees:
            residuals = tree.predict(X_new)
            leaf_ids = tree.apply(X_new)
            gamma = self._compute_gamma(residuals, leaf_ids)
            preds += self.lambd * gamma
        
        return preds

In [2]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

# Load dataset
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# Fit model
n_estimators = 100
lambd = 0.1
gb = GradientBoosting(n_estimators, lambd)
gb.fit(X, y)

In [3]:
from sklearn.metrics import mean_squared_error

# # Validate model
preds = gb.predict(X)
mean_squared_error(preds, y)

4.183580705289566e-06

## 4. Pros and cons
**Advantages:**
- Flexibility: Gradient Boosting can be applied to a wide range of machine learning problems, including regression, classification, and ranking problems.
- Ability to handle non-linear relationships: Gradient Boosting can capture complex non-linear relationships between the features and the target variable by combining the predictions of multiple decision trees.
- High accuracy: Gradient Boosting is known for its high accuracy and has been used to win many machine learning competitions.
- Feature selection: Gradient Boosting can perform feature selection implicitly by assigning a small weight to the features that are not important to the prediction.
- Reduction of bias: Bias is the presence of uncertainty or inaccuracy in machine learning results. Boosting algorithms combine multiple weak learners in a sequential method, which iteratively improves observations. This approach helps to reduce high bias that is common in machine learning models.

**Disadvantages:**
- Vulnerability to outlier data: Boosting models are vulnerable to outliers or data values that are different from the rest of the dataset. Because each model attempts to correct the faults of its predecessor, outliers can skew results significantly.
- Computational cost: Gradient Boosting is computationally expensive as it requires training a large number of decision trees.
- Overfitting: Gradient Boosting is prone to overfitting, especially when the number of trees is too large or the tree complexity is too high.
- Difficult to interpret: Gradient Boosting models are complex and can be difficult to interpret. The relationships between the features and the target variable are encoded in the structure of the decision trees, which can be difficult to understand.
- Slower prediction time: Gradient Boosting models have a slower prediction time compared to linear models, as they require evaluating the predictions of multiple decision trees.

## References
- [Gradient boosting](https://en.wikipedia.org/wiki/Gradient_boosting)
- [Gradient Tree Boosting](https://hastie.su.domains/ElemStatLearn/printings/ESLII_print12.pdf#page=373)
- [Introduction to Boosted Trees](https://xgboost.readthedocs.io/en/latest/tutorials/model.html)
- [What Is Boosting?](https://aws.amazon.com/what-is/boosting/)
- [How XGBoost Works](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-HowItWorks.html)
- [XGBoost](https://en.wikipedia.org/wiki/XGBoost)
- [An End-to-End Guide to Understand the Math behind XGBoost](https://www.analyticsvidhya.com/blog/2018/09/an-end-to-end-guide-to-understand-the-math-behind-xgboost/)
- [A Gentle Introduction to XGBoost for Applied Machine Learning](https://machinelearningmastery.com/gentle-introduction-xgboost-applied-machine-learning/)